In [1]:
# Environment setup
import os
import json
import json5
from uuid import UUID
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv(".env")  # Load your OpenAI API key from .env file

from pathlib import Path
import pathlib

from lmi import CommonLLMNames
from paperqa.settings import Settings
from aviary.envs.litqa.env import GradablePaperQAEnvironment
from ldp.agent import SimpleAgent
from ldp.alg import Evaluator, EvaluatorConfig, MeanMetricsCallback
from aviary.env import TaskDataset
from aviary.envs.litqa.task import TASK_DATASET_NAME

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/pybtex/plugin/__init__.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Set the model (OpenAI GPT-4)
llm_openai = CommonLLMNames.OPENAI_TEST.value

from paperqa.prompts import (
    CONTEXT_INNER_PROMPT,
    CONTEXT_OUTER_PROMPT,
    citation_prompt,
    default_system_prompt,
    env_reset_prompt,
    env_system_prompt,
    qa_prompt,
    select_paper_prompt,
    structured_citation_prompt,
    summary_json_prompt,
    summary_json_system_prompt,
    summary_prompt,
)
from paperqa.settings import (
    AgentSettings,
    AnswerSettings,
    IndexSettings,
    ParsingSettings,
    PromptSettings,
    Settings,
)

settings = Settings(
    llm=llm_openai,
    llm_config={
        "model_list": [
            {
                "model_name": llm_openai,
                "litellm_params": {
                    "model": llm_openai,
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    summary_llm=llm_openai,
    summary_llm_config={
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    embedding="text-embedding-3-small",
    embedding_config={},
    temperature=0.1,
    batch_size=1,
    verbosity=1,
    manifest_file=pathlib.Path.cwd().joinpath("litqa_pdfs/manifest.yaml"),
    paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
    index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
    answer=AnswerSettings(
        evidence_k=30,
        evidence_detailed_citations=True,
        evidence_retrieval=True,
        evidence_summary_length="about 100 words",
        evidence_skip_summary=False,
        answer_max_sources=15,
        max_answer_attempts=None,
        answer_length="about 200 words, but can be longer",
        max_concurrent_requests=10,
    ),
    parsing=ParsingSettings(
        chunk_size=5000,
        overlap=250,
        citation_prompt=citation_prompt,
        structured_citation_prompt=structured_citation_prompt,
    ),
    prompts=PromptSettings(
        summary=summary_prompt,
        qa=qa_prompt,
        select=select_paper_prompt,
        pre=None,
        post=None,
        system=default_system_prompt,
        use_json=True,
        summary_json=summary_json_prompt,
        summary_json_system=summary_json_system_prompt,
        context_outer=CONTEXT_OUTER_PROMPT,
        context_inner=CONTEXT_INNER_PROMPT,
    ),
    agent=AgentSettings(
        agent_llm=llm_openai,
        agent_llm_config={
            "model_list": [
                {
                    "model_name": llm_openai,
                    "litellm_params": {
                        "model": llm_openai,
                    },
                }
            ],
            "rate_limit": {
                llm_openai: "30000 per 1 minute",
            },
        },
        agent_prompt=env_reset_prompt,
        agent_system_prompt=env_system_prompt,
        search_count=8,
        index=IndexSettings(
            paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
            index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
        ),
    ),
)

In [3]:
# Load the LitQA v2 dataset (eval split)
dataset = TaskDataset.from_name(TASK_DATASET_NAME, settings=settings)

records = []
for i in range(len(dataset)):
    env = dataset.get_new_env_by_idx(i)
    q = env._query

    record = {
        "id": str(q.question_id) if isinstance(q.question_id, UUID) else q.question_id,
        "question": q.question,
        "options": q.options,
        "correct_answer": q.ideal_answer,
        "sources": env.sources,
    }
    records.append(record)

with open("data/litqa_eval_repro_v2.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved {len(records)} questions to litqa_eval_repro_v2.json")

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved 40 questions to litqa_eval_repro_v2.json


In [ ]:
from paperqa import ask

response = ask("For which of the following genes does regulation occur in the coding sequence?", settings=settings)

PaperQA version: 5.21.1.dev17+g2cf0d6c


[17:28:27] Beginning agent 'ToolSelector' run with question 'For which of the following genes does regulation occur
           in the coding sequence?' and full settings {'llm': 'gpt-4o-mini-2024-07-18', 'llm_config':              
           {'model_list': [{'model_name': 'gpt-4o-mini-2024-07-18', 'litellm_params': {'model':                    
           'gpt-4o-mini-2024-07-18', 'temperature': 0.1, 'max_tokens': 4096}}], 'rate_limit':                      
           {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'summary_llm': 'gpt-4o-mini-2024-07-18',             
           'summary_llm_config': {'rate_limit': {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'embedding':    
           'text-embedding-3-small', 'embedding_config': {}, 'temperature': 0.1, 'batch_size': 1,                  
           'texts_index_mmr_lambda': 1.0, 'verbosity': 1, 'answer': {'evidence_k': 30,                             
           'evidence_detailed_citations': True, 'evidence_retrieval': True, 'evidence_summary_length': 'about 100  
           words', 'evidence_skip_summary': False, 'answer_max_sources': 15, 'max_answer_attempts': None,          
           'answer_length': 'about 200 words, but can be longer', 'max_concurrent_requests': 10,                   
           'answer_filter_extra_background': False, 'get_evidence_if_no_contexts': True}, 'parsing': {'chunk_size':
           5000, 'page_size_limit': 1280000, 'pdfs_use_block_parsing': False, 'use_doc_details': True, 'overlap':  
           250, 'citation_prompt': 'Provide the citation for the following text in MLA Format. Do not write an     
           introductory sentence. If reporting date accessed, the current year is 2025\n\n{text}\n\nCitation:',    
           'structured_citation_prompt': "Extract the title, authors, and doi as a JSON from this MLA citation. If 
           any field can not be found, return it as null. Use title, authors, and doi as keys, author's value      
           should be a list of authors. {citation}\n\nCitation JSON:", 'disable_doc_valid_check': False,           
           'defer_embedding': False, 'chunking_algorithm': <ChunkingOptions.SIMPLE_OVERLAP: 'simple_overlap'>,     
           'doc_filters': None, 'use_human_readable_clinical_trials': False}, 'prompts': {'summary': 'Summarize the
           excerpt below to help answer a question.\n\nExcerpt from                                                
           {citation}\n\n----\n\n{text}\n\n----\n\nQuestion: {question}\n\nDo not directly answer the question,    
           instead summarize to give evidence to help answer the question. Stay detailed; report specific numbers, 
           equations, or direct quotes (marked with quotation marks). Reply "Not applicable" if the excerpt is     
           irrelevant. At the end of your response, provide an integer score from 1-10 on a newline indicating     
           relevance to question. Do not explain your score.\n\nRelevant Information Summary ({summary_length}):', 
           'qa': 'Answer the question below with the context.\n\nContext (with relevance                           
           scores):\n\n{context}\n\n----\n\nQuestion: {question}\n\nWrite an answer based on the context. If the   
           context provides insufficient information reply "I cannot answer." For each part of your answer,        
           indicate which sources most support it via citation keys at the end of sentences, like                  
           {example_citation}. Only cite from the context above and only use the citation keys from the context. ##
           Valid citation examples: \n- Example2024Example pages 3-4 \n- Example2024 pages 3-4 \n- Example2024     
           pages 3-4, Example2024 pages 5-6 \n## Invalid citation examples: \n- Example2024Example pages 3-4 and   
           pages 4-5 \n- Example2024Example (pages 3-4) \n- Example2024Example pages 3-4, pages 5-6 \n-            
           Example2024Example et al. (2024) \n- Example\

           Invalid manifest file type: .yaml

           New file to index: 10.1038%2Fs41586-019-1506-7.pdf...

[17:28:28] New file to index: 10.1038%2Fs41467-024-47207-6.pdf...

...

[17:46:57] Agent timeout after 500.0-sec, just answering.

           Generating answer for 'For which of the following genes does regulation occur in the coding sequence?'.

Encountered exception during tool call for tool gen_answer: TimeoutError("Timeout (60.00999999999663 secs): rate limit for key: ('client', 'gpt-4o-mini-2024-07-18')")


[17:48:05] Finished agent 'ToolSelector' run with question 'For which of the following genes does regulation occur 
           in the coding sequence?' and status truncated.

           Answer: 

In [5]:
async def evaluate():
    metrics_callback = MeanMetricsCallback(eval_dataset=dataset)
    evaluator = Evaluator(
        config=EvaluatorConfig(batch_size=3),
        agent=SimpleAgent(),
        dataset=dataset,
        callbacks=[metrics_callback],
    )
    await evaluator.evaluate()

    # Save result to JSON file
    with open("results/gpt_eval_results_v2.json", "w", encoding="utf-8") as f:
        json.dump(metrics_callback.eval_means, f, indent=2, ensure_ascii=False)

await evaluate()

Evaluation Iterations:   0% 0/14 [00:00<?, ?it/s]/Users/jiali03/.local/share/uv/python/cpython-3.13.5-macos-aarch64-none/lib/python3.13/asyncio/tasks.py:784: RuntimeWarning: coroutine 'map_fxn_summary' was never awaited
  def gather(*coros_or_futures, return_exceptions=False):
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS l

[17:48:34] Starting paper search for 'T cells anti-CD19 synNotch sIL-2 receptor circuit expand mouse tumor'.

           Starting paper search for 'colchicine resistance HAP1 cells gene inactivation'.

           paper_search for query 'T cells anti-CD19 synNotch sIL-2 receptor circuit expand mouse tumor' and offset
           0 returned 8 papers.

           Status: Paper Count=8 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0000

...

[21:11:22] Generating answer for 'Q: Which of these glycoRNAs does NOT show an increase in M0 macrophages upon     
           stimulation with LPS?                                                                                   
                                                                                                                   
           Options:                                                                                                
           A) U1                                                                                                   
           B) U35a                                                                                                 
           C) Y5                                                                                                   
           D) U8                                                                                                   
           E) Insufficient information to answer this question'.

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


[21:12:13] Status: Paper Count=8 | Relevant Papers=1 | Current Evidence=3 | Current Cost=$0.0091

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


[21:12:14] Completing 'Q: Which of these glycoRNAs does NOT show an increase in M0 macrophages upon stimulation    
           with LPS?                                                                                               
                                                                                                                   
           Options:                                                                                                
           A) U1                                                                                                   
           B) U35a                                                                                                 
           C) Y5                                                                                                   
           D) U8                                                                                                   
           E) Insufficient information to answer this question' as 'certain'.